Linear regression is the most popular regression model with very simple math concepts behind it. It's mostly applicable where the relation between features and a target vector can be explained linearly.
<br/><br/>
Simpliest case of a regression model is univariate (consisting of only one feature) linear regression $Y=\beta X + \epsilon$:
$$
\left[\begin{array}{c}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{array}\right]=\left[\begin{array}{cc}
1 & x_1 \\
1 & x_2 \\
\vdots & \vdots \\
1 & x_n
\end{array}\right]\left[\begin{array}{l}
\beta_0 \\
\beta_1
\end{array}\right]+\left[\begin{array}{c}
\epsilon_1 \\
\epsilon_2 \\
\vdots \\
\epsilon_n
\end{array}\right]
$$
The optimization problem is centered around finding $\beta_0$ and $\beta_1$ that minimize Sum of Squared Errors (**SSE**) (we denote it as $C$) and is called Ordinary Least Squares (**OLS**): 
$$
C=\left(\beta_0+\beta_1 x_1-y_1\right)^2+\left(\beta_0+\beta_1 x_2-y_2\right)^2+\ldots+\left(\beta_0+\beta_1 x_n-y_n\right)^2
$$
We then take a partial derivatives with respect to $\beta_0$ and $\beta_1$. $\frac{d(SSE)}{d\beta_0}=\sum\epsilon_i=0$, therefore with the best $\beta_0$ residuals are always equal to 0, therefore we can express this in an equation: $\sum y_i=N\times\beta_0+\beta_1\times\sum x_i\quad [1]$. Since this is a quadratic optimization, the extreme value of $C$ will be at its minimum. The partial derivative with respect to $\beta_1$ will be equal to:
$$
\begin{gathered}
\sum y_i=n \beta_0+\beta_1 \sum x_i \\
\sum x_i y_i=\sum x_i \beta_0+\beta_1 \sum x_i^2\quad\text{[2]}
\end{gathered}  
$$ 
Then we can find $\beta_0$ and $\beta_1$ by solving a system of equations $[1]$ and $[2]$. To make the problem more optimal we can rewrite the system of linear equations consisting of $[1]$ and $[2]$ as $X'Y=X'XB$, where:
$$
\begin{gathered}
X^{\prime} X=\left[\begin{array}{cccc}
1 & 1 & \cdots & 1 \\
x_1 & x_2 & \cdots & x_n
\end{array}\right]\left[\begin{array}{cc}
1 & x_1 \\
1 & x_2 \\
\vdots & \vdots \\
1 & x_n
\end{array}\right]=\left[\begin{array}{cc}
n & \sum x_i \\
\sum x_i & \sum x_i^2
\end{array}\right] \\
X^{\prime} Y=\left[\begin{array}{cccc}
1 & 1 & \cdots & 1 \\
x_1 & x_2 & \cdots & x_n
\end{array}\right]\left[\begin{array}{c}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{array}\right]=\left[\begin{array}{c}
\sum y_i \\
\sum x_i y_i
\end{array}\right]
\end{gathered}
$$
Given this matrix form of the regression equation we can find the vector $B=(\beta_0, \beta_1)$:
$$
\begin{gathered}
    B=(XX^{\prime})^{(-1)}X^{\prime}Y
\end{gathered}
$$

In [6]:
from sklearn import datasets
import numpy as np
import plotly.express as px

# load samle "diabetes" dataset and select only one feature
# to reduce it to univariate linear regression problem
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)
diabetes_X = diabetes_X[:, 2][:, np.newaxis]
intercept = np.ones(shape=diabetes_X.shape)
diabetes_y = diabetes_y.reshape(-1,1)

# find betas in a matrix form
B = np.linalg.inv()

px.scatter(x=diabetes_X.reshape(1,-1)[0], y=diabetes_y.reshape(1,-1)[0])
